# Ejecución del Modelo de Detección de Fraude

## Importación de librerías

In [1]:
import pandas as pd
import sys, os

# Obtiene la ruta absoluta de la carpeta raíz del proyecto
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

import funciones

## Importación de datos

In [ ]:
# Se importa la base de datos suministrada 
file_path=r"..\data\MercadoLibre.csv"
datos = pd.read_csv(
    file_path,
    sep=',',
    decimal='.'
)

# Preprocesamiento

In [3]:
# Se crea muestra de ejemplo para ejecución de prueba, solo es ejemplo, con data nueva se debe eliminar
data=datos.sample(frac=0.40, random_state=123)

In [4]:
# Selección solo de variables que se necesitan
data=funciones.seleccion_variables(data)

# Corrección de formato de datos después de la lectura
data=funciones.limpiar_y_convertir(df=data, columnas=['Monto', 'Q', 'R'])

# Corrección de valores erróneos 
data=funciones.reemplazar_negativos_por_mediana(df=data, columnas=['B', 'S'])

# Manejo de datos faltantes
data=funciones.imputar_nulos_por_mediana(df=data, columnas=['C'])

# Reconversión de variable categórica
data=funciones.dummificar_var_categorica(df=data, columna='J', categorias_principales=['AR', 'BR'], referencia='OTROS')

# Evidenciar valores poco probables a travé de valores 0/1
data=funciones.binarizar_variables(df=data, vars_corte_0=['A', 'D', 'E', 'H', 'P'], vars_corte_1=['M', 'N', 'O'])

# Disminución de efectos de escalas por medio de transformación sqrt en variables continuas
data=funciones.transformacion_raiz_cuadrada(df=data, variables=['B', 'C', 'Q', 'R', 'S', 'Monto'])

# Ejecución Modelo

## Importación del modelo

In [ ]:
model_path=r"..\models\modelo_fraude.joblib"
model=funciones.cargar_modelo(ruta_modelo=model_path)

Modelo cargado correctamente desde: \models\modelo_fraude.joblib


## Calculo de probabilidades

In [6]:
data_final=funciones.ejecutar_modelo(modelo=model,df=data, umbral=0.565)
data_final

Predicciones generadas correctamente (umbral = 0.565)


,A,B,C,D,E,H,M,N,O,P,Q,R,S,Monto,J_BR,J_AR,PROB_FRAUDE,PRED_FRAUDE
15587,0,2.828427,86.313383,0,0,0,0,0,0,1,0.000000,0.0,5.126402,5.871967,1,0,0.422014,0
1888,1,4.123106,102.313244,0,0,0,1,0,0,1,0.000000,0.0,5.794825,5.244044,0,1,0.855838,1
12229,0,1.414214,6.324555,0,0,0,1,1,0,1,25.264798,0.0,8.271638,26.111683,0,0,0.489341,0
854,1,2.828427,256.928005,0,0,0,0,0,0,1,0.000000,0.0,3.749667,12.168402,0,1,0.652210,1
5638,0,2.449490,60.695964,0,0,0,0,0,0,1,0.000000,0.0,3.794733,4.306971,0,1,0.337949,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15615,0,1.000000,55.054518,0,0,0,1,0,0,1,0.000000,0.0,6.497692,19.283672,1,0,0.185324,0
7885,0,3.741657,130.594793,0,1,0,1,0,0,1,0.000000,0.0,5.121523,6.442825,0,1,0.796543,1
7349,0,2.236068,51.807335,0,0,0,0,0,0,1,0.000000,0.0,3.106445,5.728001,0,1,0.237474,0
3376,0,1.732051,259.115804,0,0,0,0,0,0,1,0.000000,0.0,9.297849,12.027884,0,1,0.589636,1


In [11]:
print("Distribución de predicciones:")
print("\nTotales:")
print(data_final['PRED_FRAUDE'].value_counts())

print("\n%:")
print((data_final['PRED_FRAUDE'].value_counts(normalize=True)*100).round(0).astype(int).astype(str)+'%')

Distribución de predicciones:

Totales:
PRED_FRAUDE
0    4101
1    2651
Name: count, dtype: int64

%:
PRED_FRAUDE
0    61%
1    39%
Name: proportion, dtype: object
